# GTD RAG System

This notebook implements a Retrieval-Augmented Generation (RAG) system over the Getting Things Done (GTD) book chapters.

## Features
- Document loading and preprocessing from chapters/ directory
- Text chunking for optimal retrieval
- Vector embeddings using sentence-transformers
- Semantic search and retrieval
- Integration with OpenAI for generation
- Interactive query interface

In [1]:
# Install required packages
%pip install -qU sentence-transformers faiss-cpu openai python-dotenv tiktoken

# Import necessary libraries
import os
import glob
import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple
import json

# For text processing
import tiktoken
from sentence_transformers import SentenceTransformer

# For vector storage and retrieval
import faiss

# For OpenAI integration
import openai
from dotenv import load_dotenv

# For display
from IPython.display import display, Markdown

print("✅ Required packages installed and imported successfully!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.69.0 requires openai<1.76.0,>=1.68.2, but you have openai 1.93.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
✅ Required packages installed and imported successfully!


In [2]:
# Configuration
class RAGConfig:
    def __init__(self):
        self.chapters_dir = "chapters/"
        self.chunk_size = 1000
        self.chunk_overlap = 200
        self.embedding_model = "all-MiniLM-L6-v2"
        self.max_tokens = 8192
        self.top_k = 5
        
        # Load environment variables
        load_dotenv()
        
        # Set up OpenAI API key
        openai.api_key = os.getenv("OPENAI_API_KEY")
        if not openai.api_key:
            print("⚠️  Warning: OPENAI_API_KEY not found in environment variables.")
            print("   You can still use the retrieval functionality, but generation will be limited.")

config = RAGConfig()
print("✅ Configuration loaded successfully!")
print(f"   Chapters directory: {config.chapters_dir}")
print(f"   Chunk size: {config.chunk_size}")
print(f"   Embedding model: {config.embedding_model}")


✅ Configuration loaded successfully!
   Chapters directory: chapters/
   Chunk size: 1000
   Embedding model: all-MiniLM-L6-v2


In [3]:
# Document Loading
class DocumentLoader:
    def __init__(self, chapters_dir: str):
        self.chapters_dir = chapters_dir
        
    def load_documents(self) -> List[Dict[str, str]]:
        """Load all markdown documents from the chapters directory."""
        documents = []
        
        # Get all markdown files
        md_files = glob.glob(os.path.join(self.chapters_dir, "*.md"))
        
        print(f"Found {len(md_files)} markdown files to process...")
        
        for file_path in sorted(md_files):
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    
                    # Extract filename without extension
                    filename = os.path.basename(file_path)
                    
                    # Skip README file
                    if filename.lower() == 'readme.md':
                        continue
                    
                    documents.append({
                        'filename': filename,
                        'filepath': file_path,
                        'content': content,
                        'word_count': len(content.split())
                    })
                    
                print(f"✅ Loaded: {filename} ({len(content.split())} words)")
                    
            except Exception as e:
                print(f"❌ Error loading {file_path}: {str(e)}")
                
        return documents

# Load documents
loader = DocumentLoader(config.chapters_dir)
documents = loader.load_documents()

print(f"\n📚 Successfully loaded {len(documents)} documents")
print(f"   Total words: {sum(doc['word_count'] for doc in documents):,}")

# Display document summary
doc_summary = pd.DataFrame([
    {
        'Chapter': doc['filename'], 
        'Words': doc['word_count']
    } 
    for doc in documents
])
print("\n📊 Document Summary:")
display(doc_summary)


Found 16 markdown files to process...
✅ Loaded: chapter-01-the-art-of-getting-things-done.md (8819 words)
✅ Loaded: chapter-02-getting-control-of-y-our-life-the-five-steps.md (10388 words)
✅ Loaded: chapter-03-getting-projects-creatively-under-w-ay-the.md (8751 words)
✅ Loaded: chapter-04-getting-started-setting-up-the-t-ime-space.md (7915 words)
✅ Loaded: chapter-05-capturing-corralling-y-our-stuf-f.md (4628 words)
✅ Loaded: chapter-06-clarifying-getting-in-to-empty.md (6630 words)
✅ Loaded: chapter-07-or-ganizing-setting-up-the-right-buckets.md (17748 words)
✅ Loaded: chapter-08-reflecting-keeping-it-all-fresh-and.md (4667 words)
✅ Loaded: chapter-09-engaging-making-the-best-action-choices.md (8491 words)
✅ Loaded: chapter-10-getting-projects-under-control.md (4032 words)
✅ Loaded: chapter-11-the-power-of-the-capturing-habit.md (3830 words)
✅ Loaded: chapter-12-the-power-of-the-next-action-decision.md (4613 words)
✅ Loaded: chapter-13-the-power-of-outcome-focusing.md (3093 words)
✅ L

,Chapter,Words
0,chapter-01-the-art-of-getting-things-done.md,8819
1,chapter-02-getting-control-of-y-our-life-the-f...,10388
2,chapter-03-getting-projects-creatively-under-w...,8751
3,chapter-04-getting-started-setting-up-the-t-im...,7915
4,chapter-05-capturing-corralling-y-our-stuf-f.md,4628
5,chapter-06-clarifying-getting-in-to-empty.md,6630
6,chapter-07-or-ganizing-setting-up-the-right-bu...,17748
7,chapter-08-reflecting-keeping-it-all-fresh-and.md,4667
8,chapter-09-engaging-making-the-best-action-cho...,8491
9,chapter-10-getting-projects-under-control.md,4032


In [4]:
# Text Chunking
class TextChunker:
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.tokenizer = tiktoken.get_encoding("cl100k_base")
        
    def clean_text(self, text: str) -> str:
        """Clean and normalize text."""
        # Remove excessive whitespace
        text = re.sub(r'\s+', ' ', text)
        # Remove markdown headers (keep content)
        text = re.sub(r'^#+\s*', '', text, flags=re.MULTILINE)
        # Remove excessive newlines
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()
    
    def chunk_text(self, text: str, source_info: Dict[str, str]) -> List[Dict[str, str]]:
        """Split text into overlapping chunks."""
        cleaned_text = self.clean_text(text)
        
        # Split into sentences for better chunk boundaries
        sentences = re.split(r'(?<=[.!?])\s+', cleaned_text)
        
        chunks = []
        current_chunk = ""
        current_tokens = 0
        
        for sentence in sentences:
            sentence_tokens = len(self.tokenizer.encode(sentence))
            
            # If adding this sentence would exceed chunk size, save current chunk
            if current_tokens + sentence_tokens > self.chunk_size and current_chunk:
                chunks.append({
                    'text': current_chunk.strip(),
                    'source': source_info['filename'],
                    'filepath': source_info['filepath'],
                    'chunk_id': len(chunks),
                    'tokens': current_tokens
                })
                
                # Start new chunk with overlap
                if self.chunk_overlap > 0:
                    # Take last few sentences for overlap
                    overlap_sentences = current_chunk.split('. ')[-2:]
                    current_chunk = '. '.join(overlap_sentences) + '. ' + sentence
                    current_tokens = len(self.tokenizer.encode(current_chunk))
                else:
                    current_chunk = sentence
                    current_tokens = sentence_tokens
            else:
                current_chunk += " " + sentence if current_chunk else sentence
                current_tokens += sentence_tokens
        
        # Add the last chunk
        if current_chunk.strip():
            chunks.append({
                'text': current_chunk.strip(),
                'source': source_info['filename'],
                'filepath': source_info['filepath'],
                'chunk_id': len(chunks),
                'tokens': current_tokens
            })
        
        return chunks
    
    def process_documents(self, documents: List[Dict[str, str]]) -> List[Dict[str, str]]:
        """Process all documents and return chunks."""
        all_chunks = []
        
        print("🔄 Processing documents into chunks...")
        
        for doc in documents:
            chunks = self.chunk_text(doc['content'], doc)
            all_chunks.extend(chunks)
            print(f"✅ {doc['filename']}: {len(chunks)} chunks created")
        
        print(f"\n📝 Total chunks created: {len(all_chunks)}")
        print(f"   Average tokens per chunk: {np.mean([chunk['tokens'] for chunk in all_chunks]):.0f}")
        
        return all_chunks

# Create text chunks
chunker = TextChunker(config.chunk_size, config.chunk_overlap)
chunks = chunker.process_documents(documents)

# Display chunk statistics
chunk_stats = pd.DataFrame([
    {
        'Source': chunk['source'],
        'Chunk ID': chunk['chunk_id'],
        'Tokens': chunk['tokens'],
        'Preview': chunk['text'][:100] + "..." if len(chunk['text']) > 100 else chunk['text']
    }
    for chunk in chunks[:10]  # Show first 10 chunks
])

print("\n📊 Sample Chunks:")
display(chunk_stats)


🔄 Processing documents into chunks...
✅ chapter-01-the-art-of-getting-things-done.md: 12 chunks created
✅ chapter-02-getting-control-of-y-our-life-the-five-steps.md: 14 chunks created
✅ chapter-03-getting-projects-creatively-under-w-ay-the.md: 12 chunks created
✅ chapter-04-getting-started-setting-up-the-t-ime-space.md: 11 chunks created
✅ chapter-05-capturing-corralling-y-our-stuf-f.md: 7 chunks created
✅ chapter-06-clarifying-getting-in-to-empty.md: 9 chunks created
✅ chapter-07-or-ganizing-setting-up-the-right-buckets.md: 24 chunks created
✅ chapter-08-reflecting-keeping-it-all-fresh-and.md: 6 chunks created
✅ chapter-09-engaging-making-the-best-action-choices.md: 11 chunks created
✅ chapter-10-getting-projects-under-control.md: 5 chunks created
✅ chapter-11-the-power-of-the-capturing-habit.md: 5 chunks created
✅ chapter-12-the-power-of-the-next-action-decision.md: 7 chunks created
✅ chapter-13-the-power-of-outcome-focusing.md: 4 chunks created
✅ chapter-14-gtd-and-cognitive-science

,Source,Chunk ID,Tokens,Preview
0,chapter-01-the-art-of-getting-things-done.md,0,987,Chapter 1: The Art of Getting Things Done *Par...
1,chapter-01-the-art-of-getting-things-done.md,1,992,And most people are to some degree frustrated ...
2,chapter-01-the-art-of-getting-things-done.md,2,992,And if you could keep life in general more in ...
3,chapter-01-the-art-of-getting-things-done.md,3,999,(Even in the 1980s many professionals consider...
4,chapter-01-the-art-of-getting-things-done.md,4,980,The Promise: The “Ready State” of the Martial ...
5,chapter-01-the-art-of-getting-things-done.md,5,979,"Y ou probably had a sense of being in control,..."
6,chapter-01-the-art-of-getting-things-done.md,6,994,"Now , describe, in a single written sentence, ..."
7,chapter-01-the-art-of-getting-things-done.md,7,975,—Kerry Gleeson you haven’ t decided what the v...
8,chapter-01-the-art-of-getting-things-done.md,8,992,Stuf f is not inherently a bad thing. Things t...
9,chapter-01-the-art-of-getting-things-done.md,9,968,"Clarifying things on the front end, when they ..."


In [5]:
# Embedding and Vector Storage
class VectorStore:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        print(f"🔄 Loading embedding model: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.dimension = self.model.get_sentence_embedding_dimension()
        self.index = None
        self.chunks = []
        print(f"✅ Model loaded! Embedding dimension: {self.dimension}")
        
    def create_embeddings(self, chunks: List[Dict[str, str]]) -> np.ndarray:
        """Create embeddings for all chunks."""
        print("🔄 Creating embeddings...")
        
        texts = [chunk['text'] for chunk in chunks]
        
        # Create embeddings in batches to manage memory
        batch_size = 32
        embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
            batch_embeddings = self.model.encode(batch_texts, show_progress_bar=True)
            embeddings.append(batch_embeddings)
            print(f"   Processed batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}")
        
        embeddings = np.vstack(embeddings)
        print(f"✅ Created {embeddings.shape[0]} embeddings")
        
        return embeddings
    
    def build_index(self, chunks: List[Dict[str, str]]):
        """Build FAISS index from chunks."""
        self.chunks = chunks
        embeddings = self.create_embeddings(chunks)
        
        print("🔄 Building FAISS index...")
        
        # Create FAISS index
        self.index = faiss.IndexFlatIP(self.dimension)  # Inner product for cosine similarity
        
        # Normalize embeddings for cosine similarity
        faiss.normalize_L2(embeddings)
        
        # Add embeddings to index
        self.index.add(embeddings.astype('float32'))
        
        print(f"✅ FAISS index built with {self.index.ntotal} vectors")
    
    def search(self, query: str, top_k: int = 5) -> List[Dict[str, any]]:
        """Search for similar chunks."""
        if self.index is None:
            raise ValueError("Index not built. Call build_index() first.")
        
        # Create query embedding
        query_embedding = self.model.encode([query])
        faiss.normalize_L2(query_embedding)
        
        # Search
        scores, indices = self.index.search(query_embedding.astype('float32'), top_k)
        
        # Format results
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.chunks):  # Valid index
                result = self.chunks[idx].copy()
                result['similarity_score'] = float(score)
                results.append(result)
        
        return results

# Create vector store and build index
vector_store = VectorStore(config.embedding_model)
vector_store.build_index(chunks)

print(f"\n🎯 Vector store ready!")
print(f"   Index size: {vector_store.index.ntotal} vectors")
print(f"   Embedding dimension: {vector_store.dimension}")

# Test search
test_query = "What is the two-minute rule?"
test_results = vector_store.search(test_query, top_k=3)

print(f"\n🔍 Test search: '{test_query}'")
for i, result in enumerate(test_results, 1):
    print(f"\n{i}. [{result['source']}] (Score: {result['similarity_score']:.3f})")
    print(f"   {result['text'][:200]}...")


🔄 Loading embedding model: all-MiniLM-L6-v2
✅ Model loaded! Embedding dimension: 384
🔄 Creating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   Processed batch 1/5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   Processed batch 2/5


...

   Processed batch 3/5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   Processed batch 4/5


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

   Processed batch 5/5
✅ Created 139 embeddings
🔄 Building FAISS index...
✅ FAISS index built with 139 vectors

🎯 Vector store ready!
   Index size: 139 vectors
   Embedding dimension: 384

🔍 Test search: 'What is the two-minute rule?'

1. [chapter-06-clarifying-getting-in-to-empty.md] (Score: 0.485)
   That’ s a rather dramatic testimonial, but it’ s an indication of just how critical some of these simple processing behaviors can be, especially as the volume and speed of the input increase for you p...

2. [chapter-09-engaging-making-the-best-action-choices.md] (Score: 0.253)
   Y ou have three pages of scribbled notes from the conversation. There’ s a meeting scheduled with your staf f at eleven, about half an hour from now .. Y ou were out late last night with your spouse’ ...

3. [chapter-01-the-art-of-getting-things-done.md] (Score: 0.225)
   —Kerry Gleeson you haven’ t decided what the very next physical action step is; and/or you haven’ t put reminders of the outcome and the act

In [6]:
# RAG Generation System
class RAGSystem:
    def __init__(self, vector_store: VectorStore, config: RAGConfig):
        self.vector_store = vector_store
        self.config = config
        self.client = openai.OpenAI() if openai.api_key else None
        
    def retrieve_context(self, query: str, top_k: int = None) -> str:
        """Retrieve relevant context for a query."""
        if top_k is None:
            top_k = self.config.top_k
            
        results = self.vector_store.search(query, top_k)
        
        context_parts = []
        for i, result in enumerate(results, 1):
            context_parts.append(
                f"**Source {i}: {result['source']}** (Relevance: {result['similarity_score']:.3f})\n"
                f"{result['text']}\n"
            )
        
        return "\n---\n".join(context_parts)
    
    def generate_prompt(self, query: str, context: str) -> str:
        """Generate a prompt for the language model."""
        prompt = f"""You are an AI assistant specialized in David Allen's "Getting Things Done" (GTD) methodology. Use the provided context from the GTD book to answer the user's question accurately and comprehensively.

**Context from GTD book:**
{context}

**User Question:** {query}

**Instructions:**
- Answer based primarily on the provided context
- If the context doesn't fully address the question, indicate what information is missing
- Use GTD terminology and concepts accurately
- Provide practical, actionable advice when appropriate
- Maintain David Allen's tone and approach
- If you reference specific GTD principles or techniques, explain them clearly

**Answer:**"""
        return prompt
    
    def generate_answer(self, query: str, max_tokens: int = None) -> Dict[str, any]:
        """Generate an answer using RAG."""
        if max_tokens is None:
            max_tokens = self.config.max_tokens
            
        # Retrieve context
        context = self.retrieve_context(query)
        
        # Generate prompt
        prompt = self.generate_prompt(query, context)
        
        result = {
            'query': query,
            'context': context,
            'prompt': prompt,
            'answer': None,
            'error': None
        }
        
        # Generate answer if OpenAI is available
        if self.client:
            try:
                response = self.client.chat.completions.create(
                    model="gpt-4.1-mini",
                    messages=[
                        {"role": "system", "content": "You are a GTD (Getting Things Done) expert assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=max_tokens,
                    temperature=0.7
                )
                result['answer'] = response.choices[0].message.content
                
            except Exception as e:
                result['error'] = f"OpenAI API error: {str(e)}"
        else:
            result['error'] = "OpenAI API key not configured. Only retrieval available."
        
        return result
    
    def display_result(self, result: Dict[str, any], show_context: bool = False):
        """Display RAG result in a formatted way."""
        print(f"🔍 **Query:** {result['query']}")
        print("=" * 60)
        
        if result['answer']:
            print("📝 **Answer:**")
            display(Markdown(result['answer']))
        elif result['error']:
            print(f"❌ **Error:** {result['error']}")
        
        if show_context:
            print("\n📚 **Retrieved Context:**")
            print("-" * 40)
            display(Markdown(result['context']))

# Initialize RAG system
rag_system = RAGSystem(vector_store, config)

# Test the RAG system
test_questions = [
    "What is the two-minute rule in GTD?",
    "How do I organize my GTD system?",
    "What are the five steps of GTD workflow?"
]

print("🤖 Testing RAG System:")
print("=" * 50)

for question in test_questions:
    print(f"\n💭 Testing: {question}")
    result = rag_system.generate_answer(question)
    
    if result['answer']:
        print("✅ Generated answer successfully")
    else:
        print(f"⚠️  {result['error']}")
        print("📚 Retrieved context available for manual review")

print("\n🎯 RAG System is ready for use!")


🤖 Testing RAG System:

💭 Testing: What is the two-minute rule in GTD?
✅ Generated answer successfully

💭 Testing: How do I organize my GTD system?
✅ Generated answer successfully

💭 Testing: What are the five steps of GTD workflow?
✅ Generated answer successfully

🎯 RAG System is ready for use!


In [7]:
# Interactive Query Interface

def ask_gtd(question: str, show_context: bool = False, top_k: int = None):
    """
    Ask a question about GTD methodology.
    
    Args:
        question (str): Your question about GTD
        show_context (bool): Whether to display the retrieved context
        top_k (int): Number of relevant chunks to retrieve (default: 5)
    """
    if top_k:
        rag_system.config.top_k = top_k
    
    result = rag_system.generate_answer(question)
    rag_system.display_result(result, show_context=show_context)
    return result

# Demo queries
print("🎯 **GTD RAG System - Ready to Answer Your Questions!**")
print("=" * 60)
print("\n📋 **Usage Examples:**")
print("1. `ask_gtd('What is the two-minute rule?')`")
print("2. `ask_gtd('How do I do a weekly review?', show_context=True)`")
print("3. `ask_gtd('What are contexts in GTD?', top_k=3)`")

print("\n💡 **Sample Questions to Try:**")
sample_questions = [
    "What is the two-minute rule in GTD?",
    "How do I set up a GTD system?",
    "What is the difference between projects and next actions?",
    "How often should I do a weekly review?",
    "What are contexts and how do I use them?",
    "How do I capture everything in my head?",
    "What is the GTD workflow process?",
    "How do I organize my reference materials?",
    "What is the purpose of the inbox in GTD?",
    "How do I handle waiting-for items?"
]

for i, q in enumerate(sample_questions, 1):
    print(f"{i:2d}. {q}")

print(f"\n🔧 **System Status:**")
print(f"   📚 Documents loaded: {len(documents)}")
print(f"   📝 Text chunks: {len(chunks)}")
print(f"   🔍 Vector index size: {vector_store.index.ntotal}")
print(f"   🤖 OpenAI integration: {'✅ Ready' if rag_system.client else '❌ Not configured'}")

print(f"\n🚀 **Ready to answer your GTD questions!**")


🎯 **GTD RAG System - Ready to Answer Your Questions!**

📋 **Usage Examples:**
1. `ask_gtd('What is the two-minute rule?')`
2. `ask_gtd('How do I do a weekly review?', show_context=True)`
3. `ask_gtd('What are contexts in GTD?', top_k=3)`

💡 **Sample Questions to Try:**
 1. What is the two-minute rule in GTD?
 2. How do I set up a GTD system?
 3. What is the difference between projects and next actions?
 4. How often should I do a weekly review?
 5. What are contexts and how do I use them?
 6. How do I capture everything in my head?
 7. What is the GTD workflow process?
 8. How do I organize my reference materials?
 9. What is the purpose of the inbox in GTD?
10. How do I handle waiting-for items?

🔧 **System Status:**
   📚 Documents loaded: 15
   📝 Text chunks: 139
   🔍 Vector index size: 139
   🤖 OpenAI integration: ✅ Ready

🚀 **Ready to answer your GTD questions!**


In [8]:
# Demo: Try the RAG System!

# Example 1: Basic question
print("🎬 **Demo 1: Basic GTD Question**")
print("=" * 40)
ask_gtd("What is the two-minute rule in GTD?")


🎬 **Demo 1: Basic GTD Question**
🔍 **Query:** What is the two-minute rule in GTD?
📝 **Answer:**


The two-minute rule in GTD is a simple yet powerful guideline used during the "clarifying" or processing step of managing your inputs (such as emails, notes, or physical inbox items). The rule states:

**If an action you identify can be done in two minutes or less, do it immediately.**

This means when you process your "in" trays or new inputs, rather than deferring or adding it to a list, you simply complete the task right then and there. This approach helps clear many small tasks quickly, reducing clutter and mental overhead.

Here are some important details and practical advice related to the two-minute rule from the GTD context:

- **Two minutes is a guideline, not a rigid limit.** If you have more time, you might allow yourself five or ten minutes to handle quick tasks. Conversely, if you need to move faster, you can shorten it to one minute or even thirty seconds.

- **Estimate realistically.** People often underestimate how long something takes. For example, a phone call might take longer if you reach the person rather than voicemail. It helps to time yourself occasionally to get a better feel for what fits in two minutes.

- **No need to track two-minute actions.** These quick tasks are simply done and cleared without having to capture or defer them. However, if completing that quick action reveals further steps necessary to complete a project, you then clarify and organize the next action accordingly.

- **Avoid becoming a slave to two-minute actions.** The rule is primarily for processing new inputs and small tasks that arise naturally—not for spending your whole day on minor quick tasks. If a task will take longer than two minutes, ask yourself if you are the best person to do it, and if not, delegate it appropriately using a systematic handoff (such as email, notes, or direct conversation).

- **Two-minute actions can significantly improve responsiveness and productivity,** especially in environments with high volumes of email or other inputs. Often, around 30% of actionable emails can be handled this way.

In summary, the two-minute rule is a practical method to reduce overwhelm and keep your system flowing by quickly taking care of small actionable items as they arise. It supports the GTD principle of clarifying inputs immediately and prevents small tasks from lingering as open loops in your mind.

If you find yourself regularly hitting items that take longer than two minutes, you should capture them, decide the next physical, visible action, and then either do, delegate, or defer that next action according to GTD's core workflow. This keeps your system clean and your mind clear—leading to the stress-free productivity that GTD promises.

{'query': 'What is the two-minute rule in GTD?',
 'context': '**Source 1: chapter-06-clarifying-getting-in-to-empty.md** (Relevance: 0.421)\nThat’ s a rather dramatic testimonial, but it’ s an indication of just how critical some of these simple processing behaviors can be, especially as the volume and speed of the input increase for you personally . T wo minutes is in fact just a guideline.. If you have a long open window of time in which to process your in-tray , you can extend the cutof f for each item to five or ten minutes. If you’ve got to get to the bottom of all your input rapidly , in order to figure out how best to use your afternoon, then you may want to shorten the time to one minute, or even thirty seconds, so you can get through everything a little faster . Y ou’ll be surprised how many two-minute actions you can perform even on your most critical projects. It’ s not a bad idea to time yourself for a few of these while you’re becoming familiar with the process. Most peopl

In [9]:
# Demo: Show Context Retrieval

print("\n🎬 **Demo 2: Context Retrieval**")
print("=" * 40)
print("This shows how the system retrieves relevant context from the GTD chapters:")

# Example with context display
ask_gtd("How do I organize my next actions?", show_context=True, top_k=3)

print("\n" + "="*60)
print("🎉 **GTD RAG System Complete!**")
print("The system is now ready to answer questions about Getting Things Done methodology.")
print("Use the `ask_gtd()` function to query the system interactively.")



🎬 **Demo 2: Context Retrieval**
This shows how the system retrieves relevant context from the GTD chapters:
🔍 **Query:** How do I organize my next actions?
📝 **Answer:**


Organizing your Next Actions effectively is a cornerstone of the GTD workflow and personal productivity. Based on the GTD context provided, here is a comprehensive yet practical approach to organizing your Next Actions:

1. **Clarify the Next Action for Each Item**  
   When you capture something that requires action, ask yourself, “What’s the very next physical, visible activity that needs to be done to move this forward?” This next action must be concrete and specific (e.g., “Call Fred re: repair shop number” or “Draft budget-meeting agenda”). This clarity transforms vague or overwhelming tasks into manageable steps.

2. **Decide What to Do with the Next Action: Do It, Delegate It, or Defer It**  
   - **Do it:** If the action takes less than two minutes, do it immediately. This keeps your system flowing and prevents buildup.  
   - **Delegate it:** If it will take longer and you’re not the right person to do it, delegate and track it on your “Waiting For” list.  
   - **Defer it:** If it will take longer and you are the right person, defer it by placing it on a relevant Next Actions list for later execution.

3. **Use Context-Based Next Actions Lists**  
   Organize your Next Actions into lists based on context — the location, tool, or situation required to complete them. For example:  
   - “At Computer” for actions needing a computer  
   - “At Office” for actions needing your office environment  
   - “Calls” for phone calls  
   - “Errands” for things to do when out and about  
   - “Anywhere” for tasks you can do in multiple places  
   
   This allows you to focus on what you can do right now, given your current context, making your productivity much more efficient.

4. **Maintain Physical or Digital Lists as Your Trusted System**  
   Your Next Actions lists should be physically or digitally organized and reviewable. They can be:  
   - Paper lists or notebooks  
   - Software task managers or apps  
   - File folders with action items on slips of paper  
   
   The key is that your system is trusted and accessible, so you never have to keep open loops in your head.

5. **Review and Update Weekly**  
   A Weekly Review is essential to keep your Next Actions lists fresh and accurate. During this review, you confirm that each action is still relevant and that you’ve captured any new next actions that have emerged from your Projects list or new inputs.

6. **Link Next Actions to Larger Projects**  
   Since projects are defined as any desired result requiring more than one action, each project should have at least one clearly defined next action on your Next Actions lists. This ensures you always know what step moves the project forward.

7. **Capture Ideas and Non-Actionables Separately**  
   While organizing next actions, remember that some captured items may not require immediate action but need to be filed as Reference, Tickler (for future reassessment), or Trash. Keeping these categories separate prevents cluttering your Next Actions lists with things that aren’t actionable right now.

---

**Summary:**  
To organize your Next Actions, clearly define the very next physical step for each item, decide whether to do, delegate, or defer it, then place it on a context-appropriate, trusted list. Review these lists weekly to keep them current, and make sure each project has a corresponding next action. This system externalizes your commitments, freeing your mind and enabling you to engage confidently with your work.

If you want more detailed techniques on setting up specific tools or digital apps, or examples of context lists, that information is not included in the provided context but could be explored further.


📚 **Retrieved Context:**
----------------------------------------


**Source 1: chapter-02-getting-control-of-y-our-life-the-five-steps.md** (Relevance: 0.687)
That will be the stake in the ground that will keep reminding you that you have an open loop until it is finished. A W eekly Review of the list (see page 50 ) will bring this item back to you as something that’ s still outstanding.. It will stay fresh and alive in your management system (versus your head) until it is completed or eliminated. What’ s the Next Action? This is the critical question for anything you’ve captured; if you answer it appropriately , you’ll have the key substantive thing to or ganize. The “next action” is the next physical, visible activity that needs to be engaged in, in order to move the current reality of this thing toward completion. Some examples of next actions might be: Call Fred re: name and number of the repair shop he mentioned. Draft thoughts for the budget-meeting agenda. T alk to Angela about the filing system we need to set up. Research Internet for local watercolor classes. These are all real physical activities that need to happen. Reminders of these will become the primary grist for the mill of your personal productivity- management system. Do It, Delegate It, or Defer It Once you’ve decided on the next action, you have three options: 1. Do it. If an action will take less than two minutes, it should be done at the moment it is defined. 2. Delegate it. If the action will take longer than two minutes, ask yourself, Am I the right person to do this? If the answer is no, delegate it to the appropriate entity . 3. Defer it, If the action will take longer than two minutes, and you are the right person to do it, you will have to defer acting on it until later and track it on one or more “Next Actions” lists. Or ganize Being or ganized means simply that where something is matches what it means to you. The outer ring of the workflow diagram shows the eight discrete categories of reminders and materials that will result from your processing all your stuf f. T ogether they make up a total system for or ganizing just about everything that’ s on your plate, or could be added to it, on a daily and weekly basis. For nonactionable items, the possible categories are trash , incubation , and r efer ence . If no action is needed on something, you toss it, “tickle” it for later reassessment, or file it so you can find the material if you need to refer to it at another time. T o manage actionable things, you will need a list of pr ojects , storage or files for pr oject plans and materials , a calendar , a list of r eminders of next actions , and a list of r eminders of things you’r e waiting for . All of the or ganizational categories need to be physically contained in some form. When I refer to “lists,” I just mean some sort of reviewable set of reminders, which could be lists on notebook paper or in some computer program or even file folders holding separate pieces of paper for each item. For instance, the list of current projects could be kept on a page in a loose- leaf planner; it could be held in a category within the T asks function of a software application; or it could be in a simple physical file folder labeled “Projects List.” Incubating reminders (such as “After 01 March contact my accountant to set up a meeting”) may be stored in a paper -based “tickler” or “bring-forward” file or in a digital calendar application. WORKFLOW DIAGRAM—ORGANIZING Pr ojects I define a project as any desired result that can be accomplished within a year that requires more than one action step. This means that some rather small things you might not normally call projects are going to be on your Projects list, as well as some big ones. The reasoning behind my definition is that if one step won’ t complete something, some kind of goalpost needs to be set up to remind you that there’ s something still left to do. If you don’ t have a placeholder to remind you about it, it will slip back into your head. The reason for the one-year time frame is that anything you are committed to finish within that scope needs to be reviewed weekly to feel comfortable about its status. Another way to think of this is as a list of open loops, no matter what the size.

---
**Source 2: chapter-10-getting-projects-under-control.md** (Relevance: 0.636)
Perhaps you have an important meeting coming up and you know you have to prepare an agenda and materials for it. Or you’ve just inherited the job of coordinating the annual associates’ conference, and you’ve got to get it or ganized as soon as possible so you can start delegating significant pieces.. Or you need to think about and handle plans for an upcoming holiday with your family . If you haven’ t done it already , get a next action now that will start the planning process for each of these, and put it on the appropriate action list. Then proceed with further planning steps. T ypical Planning Steps The most common types of planning-oriented actions will be your own brainstorming and or ganizing, setting up meetings, and gathering information. Brainstorming Some of the projects that have your attention right now will require you to do your own free-form thinking; this is especially true of those for which you were not clear about what the next action would be when you made that decision. These should all have a next action, such as “Draft ideas re X.” Y ou need to decide where and how you want to do that action, in order to know which action list to put it on. Do you do this kind of thinking best on a computer , or by handwriting your thoughts on paper? I may choose either medium, depending on what my intuition tells me. For me this next action would go either on my “At Computer” list or on “Anywhere” (because I can draw mind maps or take rough notes wherever I am, as long as I have pen and paper). Or ganizing Y ou may have some projects for which you have already collected notes and miscellaneous support materials, and you just need to sort through them and get them into a more structured form. In this case, your next action would likely be “Or ganize Project X notes.” If you have to be in your of fice to do that (because that’ s where the files are, and you don’ t want to carry them around), that action should go on your “At Of fice” action list. If you’re carrying the project notes around with you in a folder , or in some digital device, then the “Or ganize . . .” action would go on an “Anywhere” action list if you’re going to do it by hand, or on “At Computer” if you’re going to use a word processing or outlining program, or presentation, mind-mapping, or project- planning software. One of the greatest blocks to or ganizational (and family) productivity is the lack of decision by someone about the need for a meeting, and with whom, to move something forward. Setting Up Meetings Often the next progress to be made on project thinking is to set up a meeting with the people you’d like to have involved in the brainstorming and/or decision making. That usually means sending an e-mail to the whole group or to an assistant to get it calendared, or making a phone call to the primary person to nail down a day and time. Gathering Information Sometimes the next task on project thinking is to gather more data. Maybe you need to talk to someone to get his or her input (“Call Bill re his thoughts on the managers’ meeting”). Or you need to look through the files you just inherited from last year ’ s conference (“Review Associate Conference archive files”). Or you want to surf the W eb to get a sense of what’ s happening “out there” on a new topic you’re exploring (“Look into college scholarship funds”). Random Pr oject Thinking Don’ t lose any ideas about projects that could potentially be useful. Many times you’ll think of something you don’ t want to for get when you’re in a place that has nothing to do with the project. Y ou’re driving to the store, for example, and you think of a great way to start of f the next staf f meeting. Or you’re stirring the spaghetti sauce in the kitchen and it occurs to you that you might want to give out nice tote bags to participants in the upcoming conference. Or you’re watching the evening news when you suddenly remember another key person you may want to include in the advisory council you’re putting together . If these aren’ t specifically next actions that can go directly on your action lists, you’ll still need to capture and or ganize them somewhere that makes sense. Of course, the most critical tool for ensuring that nothing gets lost is your collection system—your in-tray , pen and paper , or smartphone. Y ou need to hold all your ideas until you later decide what to do with them. T ools and Structures That Support Project Thinking No matter at what level project ideas show up, it’ s great to have good tools always close at hand for capturing them as they occur . Once they’ve been captured, it’ s useful to have access to them whenever you need to refer to them.

---
**Source 3: chapter-09-engaging-making-the-best-action-choices.md** (Relevance: 0.592)
* While Horizon 5 (purpose and principles) is obviously the most important context within which to set priorities, experience has shown me that when we understand and implement all the levels of work in which we are engaged, especially the Ground and Horizon 1 levels, we gain greater freedom and resources to do the bigger work that we’re all about. If your boat is sinking, you really don’ t care in which direction it’ s pointed!. Although a bottom-up approach is not a conceptual priority , from a practical perspective it’ s a critical factor in achieving a balanced, productive, and comfortable life. Gr ound The first thing to do is make sure your action lists are complete, which in itself can be quite a task. Those who focus on gathering and objectifying all of those items discover that there are many (often of some importance) they’ve for gotten, misplaced, or just not recognized. Aside from your calendar , if you don’ t have at least fifty next actions and waiting-fors, including all the agendas for people and meetings, I would be skeptical about whether you really had all of them. If you’ve followed through rigorously with the steps and suggestions in part 2, though, you may have them already . If not, and you do want to get this level up-to-date, set aside some time to work through chapters 4 through 6 in real implementation mode. When you’ve finished getting this level of control current, you’ll automatically have a more grounded sense of immediate priorities, which is almost impossible to achieve otherwise. Horizon 1 Finalize your Projects list. Does it truly capture all the commitments you have that will require more than one action to get done? That will define the boundaries of the kind of week-to- week operational world you’re in and allow you to relax your thinking for longer intervals. If you make a complete list of all of the things you want to have happen in your life and work at this level, you’ll discover that there are actions you need to do that you didn’ t realize. Just creating this objective inventory will give you a firmer basis on which to make decisions about what to do when you have discretionary time. Invariably when people get their Projects list up-to-date, they discover there are several things that could be done readily to move things they care about forward. T aking the inventory of your current work at all levels will automatically produce greater focus, alignment, and sense of priorities. V ery few people have this clear data defined and available to themselves in some objective form. Before any discussion about what should be done this afternoon can take place, this information must be at hand. Again, if you’ve been putting into practice the methodology of Getting Things Done , your Projects list will be where it needs to be. For most of the people we coach, it takes ten to fifteen hours of capturing, clarifying, and or ganizing to get to the point of trusting the thoroughness of their inventory . And to achieve the most pristine level of “mind like water” (nothing on your mind except what’ s present in the moment), Horizon 1 is the level that seems to incorporate some of the most interesting challenges. For all of us, there are situations and circumstances that emer ge that bother , interest, or distract us, but with which it is not immediately obvious or evident how to engage. Y our son has a problem with his math teacher; you are frustrated with how long it takes to implement a procedure in your company; you have a concern about the person running your fund-raising committee; you keep thinking you ought to be rekindling your interest in painting; etc. Quieting that subtle noise requires identifying objective outcomes for each of those (a project), with accompanying next actions placed into your trusted system. Playing consistently at this level of the stress-free productivity game is a hallmark of its mastery . Horizon 2 This is the level of “current job responsibilities” and “areas of my life to maintain at an appropriate standard.” What are the hats you wear , the roles you play? Professionally , this would relate to your current position and work. Personally , it would include the areas of responsibility you’ve taken on in your family , in your community , and of course with yourself as a functioning person. If you’re not totally sure what your job is, it will always feel overwhelming. Y ou may have some of these roles already defined and written out. If you’ve recently taken a new position and there’ s an agreement or contract about your areas of accountability , that would certainly be a good start. If you’ve done any kind of personal goal-setting and values-clarifying exercises in the past and still have any materials you created then, add those to the mix.



🎉 **GTD RAG System Complete!**
The system is now ready to answer questions about Getting Things Done methodology.
Use the `ask_gtd()` function to query the system interactively.
